<a href="https://colab.research.google.com/github/mannam95/Deep_Learning_Programming/blob/main/Assignment6/IDL_Assignment_6_Govind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

#DataLoader

In [2]:
class Dataloader:
  def __init__(self, vocab_size):
    self.vocab_size = vocab_size
    
  def preprocess(self, sequences, labels):
    return sequences, labels.astype(np.int32)
  
  def load_and_preprocess(self):
    (train_sequences, train_labels), (test_sequences, test_labels) = tf.keras.datasets.imdb.load_data(num_words=self.vocab_size)
    train_sequences, train_labels = self.preprocess(train_sequences, train_labels)
    test_sequences, test_labels = self.preprocess(test_sequences, test_labels)
    
    return train_sequences, train_labels, test_sequences, test_labels

#Helper Functions

In [5]:
def gentrain():
    for sequence, label in zip(train_sequences, train_labels):
        yield sequence, label

def gentest():
    for sequence, label in zip(test_sequences, test_labels):
        yield sequence, label


def get_train_test_data(gentrain, gentest):
  train_data = tf.data.Dataset.from_generator(gentrain, output_signature=(
          tf.TensorSpec(shape=(None,), dtype=tf.int32),
          tf.TensorSpec(shape=(), dtype=tf.int32)))
  
  test_data = tf.data.Dataset.from_generator(gentest, output_signature=(
          tf.TensorSpec(shape=(None,), dtype=tf.int32),
          tf.TensorSpec(shape=(), dtype=tf.int32)))
  return train_data, test_data

##Bucketing

In [6]:
def do_bucketing(train_data, batchSize, bucketStep, maxBucketLen):
  buckets = []
  bucketsize = 0
  while bucketsize < maxBucketLen:
    bucketsize = bucketsize + bucketStep
    buckets.append(bucketsize)
  bucket_batch_size = [batchSize] * (len(buckets) + 1)
  return train_data.bucket_by_sequence_length(lambda sequence, label: tf.shape(sequence)[0],
                                                bucket_boundaries=buckets, bucket_batch_sizes=bucket_batch_size, drop_remainder = True)

#Prepare Data

In [7]:
data_loader = Dataloader(20000)
train_sequences, train_labels, test_sequences, test_labels = data_loader.load_and_preprocess()

17473536/17464789 [==============================] - 0s 0us/step


In [8]:
train_data, test_data = get_train_test_data(gentrain, gentest)
train_data = do_bucketing(train_data, batchSize=32, bucketStep=100, maxBucketLen=1000)
test_data = do_bucketing(test_data, batchSize=32, bucketStep=100, maxBucketLen=1000)

In [9]:
# Splitting the dataset for training and testing.
def is_val(x, _):
    return x % 4 == 0


def is_train(x, y):
    return not is_val(x, y)


recover = lambda x, y: y

# Split the dataset for validation.
val_dataset = train_data.enumerate() \
    .filter(is_val) \
    .map(recover)

# Split the dataset for training
train_dataset = train_data.enumerate() \
    .filter(is_train) \
    .map(recover)

#Alpa_Model

In [17]:
alpha_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=20000,
        output_dim=32,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.LSTM(12, activation='tanh', kernel_initializer='glorot_uniform', kernel_regularizer= tf.keras.regularizers.L1L2(l1=0.01, l2=0.01)
                              ,return_sequences=True, dropout=0.2),
    tf.keras.layers.LSTM(15, activation='tanh', kernel_initializer='glorot_uniform', dropout=0.4),
    tf.keras.layers.Dense(1)
])

In [18]:
alpha_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
alpha_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 32)          640000    
                                                                 
 lstm_4 (LSTM)               (None, None, 12)          2160      
                                                                 
 lstm_5 (LSTM)               (None, 15)                1680      
                                                                 
 dense_2 (Dense)             (None, 1)                 16        
                                                                 
Total params: 643,856
Trainable params: 643,856
Non-trainable params: 0
_________________________________________________________________


In [19]:
history = alpha_model.fit(train_dataset, validation_data= val_dataset, epochs=10)

Epoch 1/10
582/582 [==============================] - 83s 120ms/step - loss: 2.6629 - accuracy: 0.5021 - val_loss: 2.1412 - val_accuracy: 0.4940
Epoch 2/10
582/582 [==============================] - 67s 114ms/step - loss: 1.6849 - accuracy: 0.6763 - val_loss: 1.3021 - val_accuracy: 0.8429
Epoch 3/10
582/582 [==============================] - 67s 115ms/step - loss: 1.0333 - accuracy: 0.8655 - val_loss: 0.8598 - val_accuracy: 0.8444
Epoch 4/10
582/582 [==============================] - 66s 114ms/step - loss: 0.6346 - accuracy: 0.9022 - val_loss: 0.6034 - val_accuracy: 0.8502
Epoch 5/10
582/582 [==============================] - 67s 115ms/step - loss: 0.4137 - accuracy: 0.9237 - val_loss: 0.4871 - val_accuracy: 0.8610
Epoch 6/10
582/582 [==============================] - 66s 113ms/step - loss: 0.3262 - accuracy: 0.9383 - val_loss: 0.4524 - val_accuracy: 0.8700
Epoch 7/10
582/582 [==============================] - 67s 114ms/step - loss: 0.2809 - accuracy: 0.9498 - val_loss: 0.4399 - val_ac

In [20]:
test_loss, test_acc = alpha_model.evaluate(test_data)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

775/775 [==============================] - 36s 47ms/step - loss: 0.5046 - accuracy: 0.8492
Test Loss: 0.5046475529670715
Test Accuracy: 0.849233865737915


#Beta_Model

In [10]:
beta_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=20000,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.LSTM(12, activation='tanh', kernel_initializer='glorot_uniform', kernel_regularizer= tf.keras.regularizers.L1L2(l1=0.01, l2=0.01)
                              ,return_sequences=True, dropout=0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(15)),

    tf.keras.layers.Dense(1)
])

In [11]:
beta_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
beta_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          1280000   
                                                                 
 lstm (LSTM)                 (None, None, 12)          3696      
                                                                 
 bidirectional (Bidirectiona  (None, 30)               3360      
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 31        
                                                                 
Total params: 1,287,087
Trainable params: 1,287,087
Non-trainable params: 0
_________________________________________________________________


In [23]:
history = beta_model.fit(train_dataset, validation_data= val_dataset, epochs=10)

Epoch 1/10
582/582 [==============================] - 84s 128ms/step - loss: 0.6676 - accuracy: 0.5309 - val_loss: 0.5198 - val_accuracy: 0.7874
Epoch 2/10
582/582 [==============================] - 72s 123ms/step - loss: 0.3911 - accuracy: 0.8427 - val_loss: 0.3549 - val_accuracy: 0.8523
Epoch 3/10
582/582 [==============================] - 72s 123ms/step - loss: 0.2784 - accuracy: 0.8916 - val_loss: 0.3070 - val_accuracy: 0.8763
Epoch 4/10
582/582 [==============================] - 72s 124ms/step - loss: 0.2272 - accuracy: 0.9155 - val_loss: 0.2907 - val_accuracy: 0.8819
Epoch 5/10
582/582 [==============================] - 72s 123ms/step - loss: 0.1998 - accuracy: 0.9254 - val_loss: 0.2888 - val_accuracy: 0.8840
Epoch 6/10
582/582 [==============================] - 72s 123ms/step - loss: 0.1557 - accuracy: 0.9489 - val_loss: 0.2997 - val_accuracy: 0.8811
Epoch 7/10
582/582 [==============================] - 71s 122ms/step - loss: 0.1339 - accuracy: 0.9575 - val_loss: 0.3079 - val_ac

In [24]:
test_loss, test_acc = beta_model.evaluate(test_data)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

775/775 [==============================] - 39s 50ms/step - loss: 0.3538 - accuracy: 0.8712
Test Loss: 0.35378602147102356
Test Accuracy: 0.8712499737739563
